### ***David Mireles Samaniego A01302935***

In [1]:
!pip install transformers

In [2]:
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from textwrap import wrap

### ***4. En este ejercicio vas a realizar una comparación simple entre un modelo Transformer preentrenado***
### ***para traducción de inglés a español, con un modelo LLM (Large Language Model).***
### ***Para ello selecciona de manera aleatoria 10 comentarios del conjunto de AmazonBaby5000.***

In [3]:
# Inicialización
RANDOM_SEED = 50
MAX_LEN = 100
BATCH_SIZE = 15
DATASET_PATH = '/content/drive/MyDrive/amazonbaby5000.csv'
NCLASSES = 2

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [4]:
# Cargar dataset
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv(DATASET_PATH)
df = df[0:5000]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
print(df.head())
print(df.shape)
print("\n".join(wrap(df['review'][200])))

                                              review  rating
0  I think it is more Expensive than drugstore th...       0
1  When I saw this on Amazon, I put it into my wi...       1
2  We really like these valances.  They have such...       1
3  No light emits from the night light. They pain...       0
4  I was really hoping for this to be a conventie...       0
(5000, 2)
I received this as a gift two years ago with my first daughter, who
loved it.  Now my second daughter has adopted it and loves it even
more.  Excellent quality, easy to clean, and both my girls love it!


In [6]:
# Reajustar dataset
df['label'] = (df['rating']=='positive').astype(int)
df.drop('rating', axis=1, inplace=True)
df.head()

,review,label
0,I think it is more Expensive than drugstore th...,0
1,"When I saw this on Amazon, I put it into my wi...",0
2,We really like these valances. They have such...,0
3,No light emits from the night light. They pain...,0
4,I was really hoping for this to be a conventie...,0


### b.- Utiliza ahora el LLM Gemini de Google a través de su API para traducir los mismos 10
### comentarios del inciso anterior. NOTA: deberás proponer el prompt que consideres
### adecuado para la traducción, incluyendo si consideras que ayuda, que tome en cuenta
### errores tipográficos (typos), o algún otro tipo de consideración.
### NOTA: Puedes consultar la siguiente liga para familiarizarte con la API de Gemini:

In [7]:
# prompt: selecciona aleatoriamente 10 comentarios de amazonbabay5000

import random

# Get all review texts
reviews = df['review'].tolist()

# Randomly select 10 reviews
random_reviews = random.sample(reviews, 10)

# Print the selected reviews
for review in random_reviews:
  print("\n".join(wrap(review)))

I ordered this blanket because my son received the same blanket a year
ago when he was born and it has become his "blankie" So, I thought I
should get another blanket incase something  happened to the
original.Unfortunately, the new blanket is a lot different from the
original and my son wants nothing to do with the new blanket. I've
tried washing it multiple times;  i've even left it in his crib, so it
would start to smell like his stuff, but the truth is the new blanket
is no where as soft and i could tell the difference as soon as I
opened the package.  The new blanket actually has a completely
different texture from the original. The fur is much shorter (almost
shaved) compared to my son's blankie which has longer fur. I don't
know if the manufacture changed how they made the blankets in the past
year or what happened, but I feel that the new blanket is a lower
quality than the original.
Love my Alva diapers , but I never got this set I totally forgot maybe
I should call  and see w

### **Traducir de Ingles a Español**

In [34]:
# prompt: traducir al español print("\n".join(wrap(review)))

from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

for review in random_reviews:
  input_ids = tokenizer.encode(review, return_tensors="pt").to(device)
  generated_tokens = model.generate(input_ids=input_ids, forced_bos_token_id=tokenizer.lang_code_to_id["es_XX"])
  translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
  print("\n".join(wrap(translated_text[0])))


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

Por lo tanto, me pareció que tenía que comprar otra tela en caso de
que se ocurriera algo con la original. Desafortunadamente, la nueva
tela es muy diferente de la original y mi hijo no quiere tener nada
que ver con la nueva tela.
Me encantan mis diapers Alva, pero nunca he recibido este conjunto he
olvidado totalmente que quizás deba llamar y ver lo que pasó :(
Es muy suave y hermosa. Compra, no te decepcionará.
Estos hacen exactamente como se anuncian pero en mi opinión no son
necesarios es simplemente algo más que tengo que limpiar y mantener
con. Mis hijos beben de sus bolsas sin problema con daños a la boca.
Lamentablemente no funcionan, a lo que nunca se le pega, madera, metal
o cristal, simplemente no se mantendrá, incluso he intentado adherirse
a ellos, mala idea!
Esta color es rojo, pero es una gran cerámica. Y sigue siendo muy
oscuro. Y yo estaba bien con un pequeño brillo de cerámica y estaba
preparado para ella basada en otras críticas. Super fácil de poner
juntos. Lo hice 

c) Despliega ambas traducciones de manera tabular y compara los resultados obtenidos.
Incluye tus conclusiones y los pros y contras que detectas en cada técnica.
NOTA: Aunque la comparación será de manera subjetiva, el punto de vista de una
persona es muy valioso y nos puede proporcionar una buena idea de los desempeños de
los modelos.

In [39]:
# prompt: from tabulate import tabulate
# matriz = [[print("\n".join(wrap(review)))],[print("\n".join(wrap(translated_text[0])))]]
# print(tabulate(matriz, headers=['Original', 'Traducido']))

from tabulate import tabulate

matriz = []
for i, review in enumerate(random_reviews):
  translated_text = tokenizer.batch_decode(model.generate(input_ids=tokenizer.encode(review, return_tensors="pt").to(device), forced_bos_token_id=tokenizer.lang_code_to_id["es_XX"]), skip_special_tokens=True)
  matriz.append([review, translated_text[0]])

print(tabulate(matriz, headers=['Original', 'Traducido']))


Original                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  Traducido
------------------------------------------------------------------------------------

In [8]:
!pip install transformers

import random
import torch
from transformers import AutoModel, AutoTokenizer
from textwrap import wrap

# Load the English-Spanish translation model using AutoClasses
model_name = "Helsinki-NLP/opus-mt-en-es"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)  # Use AutoTokenizer to automatically find the correct tokenizer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [10]:
# prompt: tokenizar con Helsinki-NLP/opus-mt-zh-en

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-zh-en")

# Tokenize the selected reviews
tokenized_reviews = tokenizer(random_reviews, padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt")

# Print the tokenized reviews
print(tokenized_reviews)

{'input_ids': tensor([[   26,     7,   672,  7876,   250,    56,  2172,  8860,  7948,    59,
            32,  6493,  4694,   158,     7,  4904,  1296,  3936,  4972,   104,
             3,    95, 18746,  2172,  8860,  7948,    59,    12,     7,   466,
         29552,    12,  6046,  7235,  1044,  1685,   169,    42,  2172, 41769,
             6,    39,  5566,  2318,    32,   148, 14829,   174,    38,   109,
          8860,  7995,   177,   478,   610,     2,    26,  9272, 14594,    59,
            26,     7,  6436, 18985,   104,   400,     7, 16689, 19114,  2172,
          8860,  7948,    59,    10,  6493,  4009,   239,   496,   102,  5566,
         15073,  1685,   250,     8,     3,     7,   672,  7201,   748,   618,
             5,  3094,   746,   589,   672,    59,  1203,  5429,   480,     0],
        [10706,   419,   158,  1825,  8619,     7, 18923,  6139,    22,     7,
             2,   183,    26,  1175,   561,    56,    95,  5270,    26,     8,
            59,  8790,    14,  6046, 

In [11]:
# Ejemplo tokenización
sample_txt = 'I really loved all the productos for baby!'
tokens = tokenizer.tokenize(sample_txt)
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print('Frase: ', sample_txt)
print('Tokens: ', tokens)
print('Tokens numéricos: ', token_ids)

Frase:  I really loved all the productos for baby!
Tokens:  ['▁I', '▁re', 'ally', '▁', 'lov', 'ed', '▁all', '▁the', '▁', 'pro', 'du', 'ct', 'os', '▁for', '▁', 'ba', 'by', '!']
Tokens numéricos:  [26, 1296, 8790, 7, 19342, 250, 58, 3, 7, 8528, 10622, 22784, 5177, 14, 7, 6099, 4449, 48]


In [12]:
# Codificación para introducir a BERT
encoding = tokenizer.encode_plus(
    sample_txt,
    max_length = 10,
    truncation = True,
    add_special_tokens = True,
    return_token_type_ids = False,
    padding = True,
    return_attention_mask = True,
    return_tensors = 'pt'
)

In [32]:
encoding.keys()

dict_keys(['input_ids', 'attention_mask'])